# **1) Initiall instructions**

In [ ]:
!pip install optuna
!pip install scikeras
!pip install rdkit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.9/78.9 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 69.8 MB/s eta 0:00:00


In [ ]:
import os
import glob
import optuna
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models

from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.metrics import accuracy_score, recall_score, matthews_corrcoef
from sklearn.model_selection import cross_val_score

from rdkit import Chem
from rdkit.Chem.AllChem import GetMorganGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **2) Data preprocessing**

In [ ]:
seed = 42
tf.keras.utils.set_random_seed(seed)

dataset = pd.read_csv('filepath') #path to file with bioactivity and smiles
df = pd.DataFrame(data=dataset, columns=['Molecule_ChEMBL_ID', 'Standard_Value', 'Smiles'])

df['Bin_Activity'] = np.where(df['Standard_Value'] <= 50, 1, 0)
df.drop(['Standard_Value'], axis='columns', inplace=True)

images_path = r"example_dataset/**/CHEMBL*/image_CHEMBL*.png" #path to directory with images.
                                                              #Due to architecture of dataset I have used glob.glob(recursive=True)
                                                              #to search recursively

images = []
labels = []
fps = []

morgan_gen = GetMorganGenerator(radius=2, fpSize=2048)

for img_path in glob.glob(images_path, recursive=True):
    img = keras.utils.load_img(img_path, target_size=(224, 224))
    x = keras.utils.img_to_array(img)
    images.append(x)

    file_name = os.path.basename(img_path)
    ch_embl_id = file_name.split('__')[0].replace("image_", "")

    matching_row = df[df['Molecule_ChEMBL_ID'] == ch_embl_id]

    if not matching_row.empty:
        label = matching_row['Bin_Activity'].values[0]
        labels.append(label)

        smiles = matching_row['Smiles'].values[0]
        mol = Chem.MolFromSmiles(smiles)
        if mol is not None:
            fingerprint = morgan_gen.GetFingerprint(mol)
            fps.append(fingerprint)
        else:
            print(f"Nie udało się wygenerować fingerprintu dla: {ch_embl_id}")
    else:
        print(f"Brak etykiety dla: {ch_embl_id}")

images_array = np.array(images) / 255.0
labels_array = np.array(labels)
fps_array = np.array([list(map(int, fp.ToBitString())) for fp in fps])

# **3) Model trainining and validation**

In [ ]:
%%time
class MultiInputKerasClassifier(BaseEstimator, ClassifierMixin):
    def __init__(self, model_fn, epochs=20, verbose=0):
        self.model_fn = model_fn
        self.epochs = epochs
        self.verbose = verbose
        self.model = None

    def fit(self, X, y):
        image_input_data = X[0]
        fingerprint_input_data = X[1]

        self.model = self.model_fn()
        self.model.fit([image_input_data, fingerprint_input_data], y, epochs=self.epochs, verbose=self.verbose)
        return self

    def predict(self, X):
        image_input_data = X[0]
        fingerprint_input_data = X[1]

        return self.model.predict([image_input_data, fingerprint_input_data])


def create_model():
    image_input = keras.layers.Input(shape=(224, 224, 3), name="image_input")
    base_model = keras.applications.DenseNet121(weights='imagenet', include_top=False, input_tensor=image_input)

    for layer in base_model.layers:
        layer.trainable = False

    x = keras.layers.GlobalAveragePooling2D()(base_model.output)
    fingerprint_input = keras.layers.Input(shape=(2048,), name="fingerprint_input")
    fingerprint_dense = keras.layers.Dense(128, activation='relu')(fingerprint_input)
    concatenated = keras.layers.Concatenate()([x, fingerprint_dense])
    dense1 = keras.layers.Dense(8, activation='relu')(concatenated)
    dropout1 = keras.layers.Dropout(0.3)(dense1)
    output = keras.layers.Dense(1, activation='sigmoid')(dropout1)

    model = keras.Model(inputs=[image_input, fingerprint_input], outputs=output)
    model.compile(optimizer=keras.optimizers.Adam(), loss='binary_crossentropy', metrics=['accuracy'])

    return model


image_input_data = images_array
fingerprint_input_data = np.array(fps_array)

X_combined = (image_input_data, fingerprint_input_data)
y = labels_array

keras_classifier = MultiInputKerasClassifier(
    model_fn=create_model,
    epochs=20,
    verbose=0
)

cv = KFold(n_splits=5, random_state=42, shuffle=True)

accuracies = []
mcc = []
recall = []

for train_index, test_index in cv.split(X_combined[0]):
    X_train_image, X_test_image = X_combined[0][train_index], X_combined[0][test_index]
    X_train_fp, X_test_fp = X_combined[1][train_index], X_combined[1][test_index]
    y_train, y_test = y[train_index], y[test_index]

    keras_classifier.fit((X_train_image, X_train_fp), y_train)
    y_pred = keras_classifier.predict((X_test_image, X_test_fp))

    accuracy_var = accuracy_score(y_test, (y_pred > 0.5).astype(int))
    accuracies.append(accuracy_var)

    mcc_var = matthews_corrcoef(y_test, (y_pred > 0.5).astype(int))
    mcc.append(mcc_var)

    recall_var = recall_score(y_test, (y_pred > 0.5).astype(int))
    recall.append(recall_var)

29084464/29084464 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ image_input (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_2          │ (None, 230, 230, 3)    │              0 │ image_input[0][0]      │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_conv (Conv2D)       │ (None, 112, 112, 64)   │          9,408 │ zero_padding2d_2[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_bn                  │ (None, 112, 112, 64)   │            256 │ conv1_conv[0][0]       │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv1_relu (Activation)   │ (None, 112, 112, 64)   │              0 │ conv1_bn[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ zero_padding2d_3          │ (None, 114, 114, 64)   │              0 │ conv1_relu[0][0]       │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ pool1 (MaxPooling2D)      │ (None, 56, 56, 64)     │              0 │ zero_padding2d_3[0][0] │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_bn         │ (None, 56, 56, 64)     │            256 │ pool1[0][0]            │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_0_relu       │ (None, 56, 56, 64)     │              0 │ conv2_block1_0_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_conv       │ (None, 56, 56, 128)    │          8,192 │ conv2_block1_0_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_bn         │ (None, 56, 56, 128)    │            512 │ conv2_block1_1_conv[0… │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_1_relu       │ (None, 56, 56, 128)    │              0 │ conv2_block1_1_bn[0][… │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_2_conv       │ (None, 56, 56, 32)     │         36,864 │ conv2_block1_1_relu[0… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block1_concat       │ (None, 56, 56, 96)     │              0 │ pool1[0][0],           │
│ (Concatenate)             │                        │                │ conv2_block1_2_conv[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2_block2_0_bn    

 Total params: 7,309,009 (27.88 MB)

 Trainable params: 271,505 (1.04 MB)

 Non-trainable params: 7,037,504 (26.85 MB)

In [ ]:
print('-----------------------------------------')
print('METRIC: ACCURACY')
for nr_acc, score_acc in enumerate(accuracies, 1):
    print(f"\tScore for split {nr_acc} is: {score_acc}")

print()
print('\tMean accuracy: ', np.mean(accuracies))
print('-----------------------------------------')


print('METRIC: MCC')
for nr_mcc, score_mcc in enumerate(mcc, 1):
    print(f"\tScore for split {nr_mcc} is: {score_mcc}")

print()
print('\tMean accuracy: ', np.mean(mcc))
print('-----------------------------------------')


print('METRIC: RECALL')
for nr_rec, score_rec in enumerate(recall, 1):
    print(f"\tScore for split {nr_rec} is: {score_rec}")

print()
print('\tMean accuracy: ', np.mean(recall))
print('-----------------------------------------')